# 1. Authenticate and Connect to BigQuery

## 1.1 Authenticate with Google Cloud

Run the following code to authenticate your Google account and allow access to BigQuery:


In [2]:
from google.colab import auth
auth.authenticate_user()



*   This will prompt you to authorize Colab to access your Google Cloud resources.

*   Follow the on-screen instructions to complete the authentication.




## 1.2 Import Libraries and Set Up BigQuery Client


In [3]:
import pandas as pd
import numpy as np
from google.cloud import bigquery


## 1.3 Set Up BigQuery Client with Your Project ID

In [4]:
# Replace 'lunar-listener-438315-b9' with your actual Google Cloud project ID
project_id = 'lunar-listener-438315-b9'

# Create a BigQuery client
client = bigquery.Client(project=project_id)


# 2. Query the Presentation Table

## 2.1 Define the SQL Query

In [5]:
# Define the SQL query to retrieve data from the presentation table
query = """
SELECT
  *
FROM `lunar-listener-438315-b9.resq_data.presentation_table`
"""


## 2.2 Execute the Query and Load the Data into a DataFrame

In [6]:
# Execute the query and convert the results to a pandas DataFrame
df = client.query(query).to_dataframe()


## 2.3 Verify the DataFrame

In [7]:
# Display the first few rows of the DataFrame
df.head()


,order_id,order_created_at,user_id,quantity,refunded,currency,sales,provider_id,defaultoffertype,provider_country,provider_registered_date,user_country,user_registered_date
0,5416827086004895733,2022-11-25 05:07:20+00:00,6440745749093560506,1,True,eur,300.0,1713020136823136256,grocery-bag,fin,2018-10-25 14:13:17+00:00,FI,2018-08-16 16:58:06+00:00
1,2717676373470635754,2023-08-15 04:15:56+00:00,4939382371357575364,1,True,eur,480.0,1713020136823136256,grocery-bag,fin,2018-10-25 14:13:17+00:00,FI,2016-06-11 15:03:48+00:00
2,2386177465838996792,2022-11-28 05:01:08+00:00,2333573918598501663,1,True,eur,500.0,1713020136823136256,grocery-bag,fin,2018-10-25 14:13:17+00:00,FI,2016-09-05 14:45:31+00:00
3,1354343446370302188,2023-07-12 12:48:57+00:00,3460397126179069157,1,True,eur,120.0,1713020136823136256,grocery-bag,fin,2018-10-25 14:13:17+00:00,FI,2016-06-21 10:39:47+00:00
4,5055826703931167592,2022-11-28 06:13:05+00:00,5558542277679015545,1,True,eur,500.0,1713020136823136256,grocery-bag,fin,2018-10-25 14:13:17+00:00,FI,2022-10-08 12:22:01+00:00


# 3. Perform the CLV Analysis

Now that we have the data in a pandas DataFrame (`df`), we can proceed with the CLV analysis.

## 3.1 Data Preparation

### 3.1.1 Convert Date columns to Datetime

In [8]:
# Ensure date columns are in datetime format
df['order_created_at'] = pd.to_datetime(df['order_created_at'])
df['user_registered_date'] = pd.to_datetime(df['user_registered_date'])
df['provider_registered_date'] = pd.to_datetime(df['provider_registered_date'])


### 3.1.2 Replace Empty strings with `NaN`

In [9]:
# Identify columns with object data type (typically strings)
string_columns = df.select_dtypes(include=['object']).columns.tolist()
print("String-type columns:", string_columns)



String-type columns: ['currency', 'defaultoffertype', 'provider_country', 'user_country']


### 3.1.3 Check for Missing Values

In [10]:
# Replace empty strings with NaN in all string-type columns
for col in string_columns:
    df[col] = df[col].replace(r'^\s*$', np.nan, regex=True)

# Check for missing values
print(df.isna().sum())



order_id                    0
order_created_at            0
user_id                     0
quantity                    0
refunded                    0
currency                    0
sales                       0
provider_id                 0
defaultoffertype            0
provider_country            0
provider_registered_date    0
user_country                3
user_registered_date        0
dtype: int64


Since we only have 3 NaN values for user_country we do not need to remove them as the rows are still useful for providing us other insights, we can simply remove the rows when looking at region specific analysis.

## 3.2 Calculate Key Metrics

### 3.2.1 Average Order Value (AOV)

In [49]:
# Calculate Total Revenue and Total Orders
total_revenue = df['sales'].sum()
total_orders = df['order_id'].nunique()

# Calculate Average Order Value
aov = total_revenue / total_orders

print(f"Total Revenue: ${total_revenue:.2f}")
print(f"Total Orders: {total_orders}")
print(f"Average Order Value (AOV): ${aov:.2f}")


Total Revenue: $231086039.00
Total Orders: 299971
Average Order Value (AOV): $770.36


### 3.2.2 Average Customer Lifespan

In [52]:
# Average Customer Lifespan
customer_lifespans = df.groupby('user_id')['order_created_at'].agg(['min', 'max'])
customer_lifespans['lifespan_days'] = (customer_lifespans['max'] - customer_lifespans['min']).dt.days
customer_lifespans['lifespan_days'] = customer_lifespans['lifespan_days'].replace(0, 1)  # Handle single purchases
average_lifespan_days = customer_lifespans['lifespan_days'].mean()
average_lifespan_years = average_lifespan_days / 365


### 3.2.3 Purchase Frequency

In [51]:
# Calculate Total Customers
total_customers = df['user_id'].nunique()

# Calculate Purchase Frequency
purchase_frequency = total_orders / total_customers

print(f"Total Customers: {total_customers}")
print(f"Purchase Frequency: {purchase_frequency:.2f}")


Total Customers: 123208
Purchase Frequency: 2.43


## 3.3 Calculate Customer Lifetime Value (CLV)

In [53]:
# Calculate CLV
clv = aov * purchase_frequency * average_lifespan_years

print(f"Expected Customer Lifetime Value (CLV): ${clv:.2f}")


Expected Customer Lifetime Value (CLV): $398.36


# 4. Summary of Findings

In [55]:
# Display the results
print("=== CLV Analysis Summary ===")
print(f"Total Revenue: ${total_revenue:.2f}")
print(f"Total Orders: {total_orders}")
print(f"Total Customers: {total_customers}")
print(f"Average Order Value (AOV): ${aov:.2f}")
print(f"Purchase Frequency: {purchase_frequency:.2f}")
print(f"Average Customer Lifespan: {average_lifespan_years:.2f} years")
print(f"Expected Customer Lifetime Value (CLV): ${clv:.2f}")

=== CLV Analysis Summary ===
Total Revenue: $231086039.00
Total Orders: 299971
Total Customers: 123208
Average Order Value (AOV): $770.36
Purchase Frequency: 2.43
Average Customer Lifespan: 0.212392208 years
Expected Customer Lifetime Value (CLV): $398.36
